In [1]:
import pandas as pd

**Задание 1:** <br>
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [2]:
# Если я правильно понял, откуда взять инфу по фильмам :-)
ratings = pd.read_csv("ml-latest-small/ratings.csv")
movies = pd.read_csv("ml-latest-small/movies.csv")

In [3]:
def movie_rank(value):
    if value <= 2:
        return 'низкий рейтинг'
    elif value <= 4:
        return 'средний рейтинг'
    elif value <= 5:
        return 'высокий рейтинг'
    return 'нет значения'

In [4]:
# простой вариант с отбором только по rating
ratings['class'] = ratings['rating'].apply(movie_rank)
ratings.head()

,userId,movieId,rating,timestamp,class
0,1,1,4.0,964982703,средний рейтинг
1,1,3,4.0,964981247,средний рейтинг
2,1,6,4.0,964982224,средний рейтинг
3,1,47,5.0,964983815,высокий рейтинг
4,1,50,5.0,964982931,высокий рейтинг


In [5]:
# через среднее и join с movies.csv

# находим средние рейтинги фильтов
avg_ratings = (
    ratings
    .groupby('movieId')
    .mean()
    .reset_index()[['movieId','rating']]
)

# объединяем справочник фильмов с его средним рейтингом и применяем функцию парсинга movie_rank
movies['class'] = movies.merge(avg_ratings, how = 'left', on = 'movieId')['rating'].apply(movie_rank)
movies.head()

,movieId,title,genres,class
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,средний рейтинг
1,2,Jumanji (1995),Adventure|Children|Fantasy,средний рейтинг
2,3,Grumpier Old Men (1995),Comedy|Romance,средний рейтинг
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,средний рейтинг
4,5,Father of the Bride Part II (1995),Comedy,средний рейтинг


**Задание 2** <br>
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:
```
geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}
```
Результат классификации запишите в отдельный столбец region.

In [6]:
keywords = pd.read_csv('keywords.csv')

In [7]:
geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

In [8]:
# через строку
def define_region_way_1(row): 
    result = set()
    for key, values in geo_data.items():
        for value in values:
            if value in row['keyword'].lower():
                result.add(key)
    if len(result) == 0:
        return 'undefined'
    else:
        return ', '.join(result)

In [9]:
# через таблицу
def define_region_way_2(row):
    result = set()
    for key, values in geo_data.items():
        for value in values:
            if value in row.lower():
                result.add(key)
    if len(result) == 0:
        return 'undefined'
    else:
        return ', '.join(result)

In [10]:
# Варинат 1
keywords['region'] = keywords.apply(define_region_way_1,axis = 1)

# Варинат 2
keywords['region2'] = keywords['keyword'].apply(define_region_way_2)

# Для проверки
keywords[['keyword','shows','region']][keywords['region'] != 'undefined'].head(10)

,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
564,погода в санкт петербурге,292489,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток
1236,банк санкт петербург,174375,Северо-Запад
1259,москва,153050,Центр
1346,санкт петербург,144232,Северо-Запад
1396,гисметео москва,140279,Центр
1482,госуслуги москва,133061,Центр


**Задание 3 (бонусное)** <br>

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
    1. для каждой строки пройдите по всем годам списка years
    2. если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    3. если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’
4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [11]:
# Если я правильно понял, откуда взять инфу по фильмам :-)
ratings = pd.read_csv("ml-latest-small/ratings.csv")
movies = pd.read_csv("ml-latest-small/movies.csv")

In [12]:
def generate_years_list(from_year, to_year):
    
    # Можно проста range(1950,2019)
    
    result = []
    for i in range(to_year - from_year+1):
        result.append(from_year + i)
    return result        

In [13]:
def production_year(row, years):
    for year in years:
        # Возвращаем первую запись вида (YYYY)
        if f'({year})' in row:
            return year
    # Если запись из диапазона не найдена, то возвращаем 1900
    return 1900

In [14]:
list_of_years = generate_years_list(1950, 2010)
movies['year'] = movies['title'].apply(production_year, years = list_of_years)
# Join на выставленные юзерами рейтинги и фильтр только по нужным столцам
result = movies.merge(ratings, how = 'left', on = 'movieId')[['year', 'rating']]
# Группировка 
(result
     .groupby('year') # по году
     .mean() # берем среднее по оценкам пользователей
     .sort_values(by = 'rating', ascending = False) # сортировка
     .reset_index() # убираем индекс из года
)

,year,rating
0,1957,4.039535
1,1954,4.009191
2,1962,3.969466
3,1952,3.953125
4,1964,3.940160
5,1972,3.939891
6,1974,3.935622
7,1967,3.922572
8,1975,3.879121
9,1958,3.842424
